In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))

library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(tibble, warn.conflicts = FALSE)

source("src/paths/paths.R")
source("src/load/load.R")


Linking to GEOS 3.12.0, GDAL 3.7.3, PROJ 9.3.0; sf_use_s2() is TRUE

terra 1.7.55



In [2]:
start_date <- as.Date("2000-01-01")
brun <- concat_tables(
    read_parquet("cache/series/BRUN/T_MAX/qc_era5.parquet", as_data_frame = FALSE) |> filter(date >= start_date) |> mutate(variable = "T_MAX", flavor = "qc_era5") |> left_join(read_parquet("cache/metadata/BRUN/T_MAX/qc_era5.parquet", as_data_frame = FALSE) |> select(identifier, section = region_)) |> rename(value = T_MAX) |> as_arrow_table(),
    read_parquet("cache/series/BRUN/T_MIN/qc_era5.parquet", as_data_frame = FALSE) |> filter(date >= start_date) |> mutate(variable = "T_MIN", flavor = "qc_era5") |> left_join(read_parquet("cache/metadata/BRUN/T_MIN/qc_era5.parquet", as_data_frame = FALSE) |> select(identifier, section = region_)) |> rename(value = T_MIN) |> as_arrow_table(),
    read_parquet("cache/series/BRUN/T_MAX/qc_homo.parquet", as_data_frame = FALSE) |> filter(date >= start_date) |> mutate(variable = "T_MAX", flavor = "qc_homo") |> left_join(read_parquet("cache/metadata/BRUN/T_MAX/qc_homo.parquet", as_data_frame = FALSE) |> select(identifier, section = region_)) |> rename(value = T_MAX) |> as_arrow_table(),
    read_parquet("cache/series/BRUN/T_MIN/qc_homo.parquet", as_data_frame = FALSE) |> filter(date >= start_date) |> mutate(variable = "T_MIN", flavor = "qc_homo") |> left_join(read_parquet("cache/metadata/BRUN/T_MIN/qc_homo.parquet", as_data_frame = FALSE) |> select(identifier, section = region_)) |> rename(value = T_MIN) |> as_arrow_table(),
    read_parquet("cache/series/BRUN/T_MAX/raw.parquet", as_data_frame = FALSE) |> filter(date >= start_date) |> mutate(variable = "T_MAX", flavor = "raw") |> left_join(read_parquet("cache/metadata/BRUN/T_MAX/raw.parquet", as_data_frame = FALSE) |> select(identifier, section = region_)) |> rename(value = T_MAX) |> as_arrow_table(),
    read_parquet("cache/series/BRUN/T_MIN/raw.parquet", as_data_frame = FALSE) |> filter(date >= start_date) |> mutate(variable = "T_MIN", flavor = "raw") |> left_join(read_parquet("cache/metadata/BRUN/T_MIN/raw.parquet", as_data_frame = FALSE) |> select(identifier, section = region_)) |> rename(value = T_MIN) |> as_arrow_table(),
) |> filter(!is.na(value)) |> mutate(
        identifier = cast(identifier, utf8()),
        value = cast(value, float32()),
        variable = cast(variable, utf8()),
        flavor = cast(flavor, utf8()),
        section = cast(section, utf8())
)


In [3]:
brun |>
    mutate(dpc = (section == "dpc")) |>
    select(-section) |>
    collect() |>
    arrange(flavor, variable, dpc, identifier, date) |>
    write_dataset(file.path("db", "data", "brun"), partitioning = c("flavor", "variable"), format = "feather")


In [2]:
brun <- concat_tables(
    read_parquet("cache/metadata/BRUN/T_MAX/qc_era5.parquet", as_data_frame = FALSE) |> mutate(variable = "T_MAX", flavor = "qc_era5") |> as_arrow_table(),
    read_parquet("cache/metadata/BRUN/T_MIN/qc_era5.parquet", as_data_frame = FALSE) |> mutate(variable = "T_MIN", flavor = "qc_era5") |> as_arrow_table(),
    read_parquet("cache/metadata/BRUN/T_MAX/qc_homo.parquet", as_data_frame = FALSE) |> mutate(variable = "T_MAX", flavor = "qc_homo") |> as_arrow_table(),
    read_parquet("cache/metadata/BRUN/T_MIN/qc_homo.parquet", as_data_frame = FALSE) |> mutate(variable = "T_MIN", flavor = "qc_homo") |> as_arrow_table(),
    read_parquet("cache/metadata/BRUN/T_MAX/raw.parquet", as_data_frame = FALSE) |> mutate(variable = "T_MAX", flavor = "raw") |> as_arrow_table(),
    read_parquet("cache/metadata/BRUN/T_MIN/raw.parquet", as_data_frame = FALSE) |> mutate(variable = "T_MIN", flavor = "raw") |> as_arrow_table(),
)


In [3]:
brun

Table
22816 rows x 16 columns
$region_ <dictionary<values=string, indices=int32>>
$country <dictionary<values=string, indices=int32>>
$anagrafica <string>
$GSOD <bool>
$user_code <string>
$MG <bool>
$identifier <string>
$lon <double>
$lat <double>
$elevation <double>
$province <dictionary<values=string, indices=int32>>
$version <int32>
$internal_id <int32>
$state <string>
$variable <string>
$flavor <string>

In [5]:
open.dataset("DPC", "metadata")

FileSystemDataset (query)
country: dictionary<values=string, indices=int32>
anagrafica: string
GSOD: bool
user_code: string
MG: bool
identifier: string
lon: double
lat: double
elevation: double
province: dictionary<values=string, indices=int32>
version: int32
internal_id: int32
state: string
dpc: bool
flavor: string
variable: string

* Filter: dpc
See $.data for the source Arrow object